In [1]:
!wget https://data.cityofnewyork.us/api/views/hyij-8hr7/rows.csv?accessType=DOWNLOAD

--2016-11-10 17:30:20--  https://data.cityofnewyork.us/api/views/hyij-8hr7/rows.csv?accessType=DOWNLOAD
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.68.26
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [ <=>                  ] 185.12M  2.15MB/s   in 95s    

Last-modified header invalid -- time-stamp ignored.
2016-11-10 17:31:55 (1.96 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [194111037]



In [2]:
!mv rows.csv?accessType=DOWNLOAD crime.csv

In [3]:
!csvcut crime.csv | head -10 | csvlook

|-----------+------------+------------------------+-------------+------------------+----------------+-----------------+-----------------+----------------+--------------+---------------+--------------------------------+------------------------+--------+----------+-----------+---------------------+-------------+-------------+----------------------------------------|
|  OBJECTID | Identifier | Occurrence Date        | Day of Week | Occurrence Month | Occurrence Day | Occurrence Year | Occurrence Hour | CompStat Month | CompStat Day | CompStat Year | Offense                        | Offense Classification | Sector | Precinct | Borough   | Jurisdiction        | XCoordinate | YCoordinate | Location 1                             |
|-----------+------------+------------------------+-------------+------------------+----------------+-----------------+-----------------+----------------+--------------+---------------+--------------------------------+------------------------+--------+----------+---

In [4]:
!wc -l crime.csv

1123466 crime.csv


In [5]:
!csvcut crime.csv | csvgrep -c12 -m 'RAPE' > rape.csv

In [6]:
!csvcut -n rape.csv 

  1: OBJECTID
  2: Identifier
  3: Occurrence Date
  4: Day of Week
  5: Occurrence Month
  6: Occurrence Day
  7: Occurrence Year
  8: Occurrence Hour
  9: CompStat Month
 10: CompStat Day
 11: CompStat Year
 12: Offense
 13: Offense Classification
 14: Sector
 15: Precinct
 16: Borough
 17: Jurisdiction
 18: XCoordinate
 19: YCoordinate
 20: Location 1


In [7]:
!wc -l rape.csv

13780 rape.csv


In [8]:
!csvcut -c3,4,5,6,7,8,12,14,15,16,17,18,19 rape.csv > rape_final.csv

In [9]:
!csvcut rape_final.csv | head | csvlook

|-------------------------+-------------+------------------+----------------+-----------------+-----------------+---------+--------+----------+-----------+---------------------+-------------+--------------|
|  Occurrence Date        | Day of Week | Occurrence Month | Occurrence Day | Occurrence Year | Occurrence Hour | Offense | Sector | Precinct | Borough   | Jurisdiction        | XCoordinate | YCoordinate  |
|-------------------------+-------------+------------------+----------------+-----------------+-----------------+---------+--------+----------+-----------+---------------------+-------------+--------------|
|  08/08/1994 06:00:00 PM | Monday      | Aug              | 8              | 1994            | 18              | RAPE    | A      | 103      | QUEENS    | N.Y. POLICE DEPT    | 1041749     | 196938       |
|  02/01/1995 08:00:00 AM | Wednesday   | Feb              | 1              | 1995            | 8               | RAPE    | G      | 83       | BROOKLYN  | N.Y. POLICE DEPT

In [10]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [11]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [12]:
!createdb -U dbuser Project2

In [13]:
%sql postgresql://dbuser@localhost:5432/Project2

'Connected: dbuser@Project2'

In [14]:
!csvcut -n rape_final.csv

  1: Occurrence Date
  2: Day of Week
  3: Occurrence Month
  4: Occurrence Day
  5: Occurrence Year
  6: Occurrence Hour
  7: Offense
  8: Sector
  9: Precinct
 10: Borough
 11: Jurisdiction
 12: XCoordinate
 13: YCoordinate


In [15]:
!csvstat rape_final.csv

  1. Occurrence Date
	<class 'datetime.datetime'>
	Nulls: True
	Min: 1975-01-01 00:01:00
	Max: 2015-12-30 18:00:00
	Unique values: 12558
	5 most frequent values:
		2005-01-01 00:01:00:	23
		2007-01-01 00:01:00:	22
		2008-01-01 00:01:00:	20
		2006-01-01 00:01:00:	18
		2010-01-01 00:01:00:	18
  2. Day of Week
	<class 'str'>
	Nulls: True
	Unique values: 7
	5 most frequent values:
		Saturday:	2312
		Sunday:	2255
		Friday:	2067
		Monday:	1831
		Thursday:	1776
	Max length: 9
  3. Occurrence Month
	<class 'str'>
	Nulls: True
	Unique values: 12
	5 most frequent values:
		Jan:	1414
		Jul:	1299
		Aug:	1268
		Jun:	1266
		Oct:	1166
	Max length: 4
  4. Occurrence Day
	<class 'int'>
	Nulls: True
	Min: 1
	Max: 31
	Sum: 190088
	Mean: 13.79648715343301
	Median: 14.0
	Standard Deviation: 9.558368831423818
	Unique values: 31
	5 most frequent values:
		1:	2178
		15:	504
		20:	429
		10:	428
		25:	424
  5. Occurrence Year
	<class 'int'>
	Nulls: True
	Min: 1975
	Max: 2015
	Sum: 27688816
	Mean: 2009.931475029

In [16]:
%%sql
DROP TABLE IF EXISTS rapecrimes;
CREATE TABLE rapecrimes (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5),
    offense CHAR(5),
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19),
    xcor BIGINT,
    ycor BIGINT
    )


Done.
Done.


[]

In [17]:
%%sql
COPY rapecrimes FROM '/home/jovyan/work/rape_final.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [18]:
%%sql
ALTER TABLE rapecrimes
ADD COLUMN TIME_ID SERIAL PRIMARY KEY;

Done.


[]

In [19]:
%%sql
select * from rapecrimes
LIMIT 10;

10 rows affected.


[(datetime.datetime(1994, 8, 8, 18, 0), 'Monday   ', 'Aug ', 8, 1994, '18   ', 'RAPE ', 'A   ', 103, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 1041749, 196938, 1),
 (datetime.datetime(1995, 2, 1, 8, 0), 'Wednesday', 'Feb ', 1, 1995, '8    ', 'RAPE ', 'G   ', 83, 'BROOKLYN           ', 'N.Y. POLICE DEPT   ', 1006965, 193571, 2),
 (datetime.datetime(2000, 1, 9, 20, 0), 'Sunday   ', 'Jan ', 9, 2000, '20   ', 'RAPE ', 'G   ', 108, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 996793, 210001, 3),
 (datetime.datetime(2001, 1, 9, 15, 0), 'Tuesday  ', 'Jan ', 9, 2001, '15   ', 'RAPE ', 'H   ', 63, 'BROOKLYN           ', 'N.Y. HOUSING POLICE', 1000448, 168068, 4),
 (datetime.datetime(2001, 2, 11, 0, 1), 'Sunday   ', 'Feb ', 11, 2001, '0    ', 'RAPE ', 'B   ', 25, 'MANHATTAN          ', 'N.Y. POLICE DEPT   ', 1000555, 230994, 5),
 (datetime.datetime(2001, 11, 19, 12, 0), 'Monday   ', 'Nov ', 19, 2001, '12   ', 'RAPE ', 'G   ', 75, 'BROOKLYN           ', 'N.Y. POLICE DEPT   ', 1017141, 183798, 6),
 (datetime.datetime(2002, 7, 25, 12, 0), 'Thursday ', 'Jul ', 25, 2002, '12   ', 'RAPE ', 'A   ', 101, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 1053678, 159044, 7),
 (datetime.datetime(2002, 7, 30, 12, 0), 'Tuesday  ', 'Jul ', 30, 2002, '12   ', 'RAPE ', 'E   ', 47, 'BRONX              ', 'N.Y. POLICE DEPT   ', 1026387, 262634, 8),
 (datetime.datetime(2002, 10, 15, 7, 0), 'Tuesday  ', 'Oct ', 15, 2002, '7    ', 'RAPE ', 'C   ', 79, 'BROOKLYN           ', 'N.Y. POLICE DEPT   ', 999634, 190253, 9),
 (datetime.datetime(2002, 11, 25, 0, 1), 'Monday   ', 'Nov ', 25, 2002, '0    ', 'RAPE ', 'B   ', 17, 'MANHATTAN          ', 'N.Y. POLICE DEPT   ', 992411, 215025, 10)]

In [20]:
%%sql
select year,count(*) from rapecrimes
GROUP BY year
LIMIT 10;

10 rows affected.


[(None, 3),
 (1975, 1),
 (1992, 3),
 (2008, 1255),
 (2004, 115),
 (1995, 5),
 (2014, 1204),
 (1994, 6),
 (1989, 2),
 (2012, 1372)]

In [21]:
%%sql
select day,count(*) from rapecrimes
GROUP BY day


8 rows affected.


[(None, 1),
 ('Thursday ', 1776),
 ('Wednesday', 1769),
 ('Monday   ', 1831),
 ('Tuesday  ', 1768),
 ('Sunday   ', 2255),
 ('Friday   ', 2067),
 ('Saturday ', 2312)]

In [22]:
%%sql
select month,count(*) from rapecrimes
GROUP BY month

13 rows affected.


[(None, 1),
 ('Mar ', 1060),
 ('Apr ', 1031),
 ('Oct ', 1166),
 ('Nov ', 1077),
 ('Jul ', 1299),
 ('Aug ', 1268),
 ('Sep ', 1159),
 ('May ', 1085),
 ('Jan ', 1414),
 ('Feb ', 921),
 ('Jun ', 1266),
 ('Dec ', 1032)]

In [23]:
#problem 3

In [29]:
#1. dimension table for time

In [30]:
%%sql
DROP TABLE IF EXISTS time_table;
CREATE TABLE time_table (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5)
    )

Done.
Done.


[]

In [31]:
!csvcut -c1,2,3,4,5,6 rape_final.csv > time.csv

In [32]:
!csvcut -n time.csv

  1: Occurrence Date
  2: Day of Week
  3: Occurrence Month
  4: Occurrence Day
  5: Occurrence Year
  6: Occurrence Hour


In [33]:
%%sql
COPY time_table FROM '/home/jovyan/work/time.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [34]:
%%sql
ALTER TABLE time_table
ADD COLUMN TIME_ID SERIAL PRIMARY KEY;

Done.


[]

In [35]:
%%sql
select * from time_table
LIMIT 5;

5 rows affected.


[(datetime.datetime(1994, 8, 8, 18, 0), 'Monday   ', 'Aug ', 8, 1994, '18   ', 1),
 (datetime.datetime(1995, 2, 1, 8, 0), 'Wednesday', 'Feb ', 1, 1995, '8    ', 2),
 (datetime.datetime(2000, 1, 9, 20, 0), 'Sunday   ', 'Jan ', 9, 2000, '20   ', 3),
 (datetime.datetime(2001, 1, 9, 15, 0), 'Tuesday  ', 'Jan ', 9, 2001, '15   ', 4),
 (datetime.datetime(2001, 2, 11, 0, 1), 'Sunday   ', 'Feb ', 11, 2001, '0    ', 5)]

In [36]:
#2. area table

In [37]:
!csvcut -c8,9,10,11 rape_final.csv > area.csv 

In [38]:
%%sql
DROP TABLE IF EXISTS area_table;
CREATE TABLE area_table (
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19)
    )

Done.
Done.


[]

In [39]:
%%sql
COPY area_table FROM '/home/jovyan/work/area.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [40]:
%%sql
ALTER TABLE area_table
ADD COLUMN area_ID SERIAL PRIMARY KEY;

Done.


[]

In [41]:
%%sql
select * from area_table
LIMIT 5;

5 rows affected.


[('A   ', 103, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 1),
 ('G   ', 83, 'BROOKLYN           ', 'N.Y. POLICE DEPT   ', 2),
 ('G   ', 108, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 3),
 ('H   ', 63, 'BROOKLYN           ', 'N.Y. HOUSING POLICE', 4),
 ('B   ', 25, 'MANHATTAN          ', 'N.Y. POLICE DEPT   ', 5)]

In [42]:
#3. fact table

In [50]:
%%sql
DROP TABLE IF EXISTS fact_table;
CREATE TABLE fact_table (
    time_ID int4,
    area_ID int4,
    offense
    FOREIGN KEY (time_ID) REFERENCES time_table(time_id),
    FOREIGN KEY (area_ID) REFERENCES area_table(area_id)
    )

Done.
Done.


[]

In [51]:
%%sql
select * from fact_table
LIMIT 5;

0 rows affected.


[]

# Problem 3

1.Now we need to create a table to store all data. 

In [24]:
%%sql
DROP TABLE IF EXISTS rapecrimes;
CREATE TABLE rapecrimes (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5),
    offense CHAR(5),
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19),
    xcor BIGINT,
    ycor BIGINT
    )

Done.
Done.


[]

In [25]:
%%sql
COPY rapecrimes FROM '/home/jovyan/work/rape_final.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [26]:
%%sql
ALTER TABLE rapecrimes
ADD COLUMN TIME_ID SERIAL PRIMARY KEY;

Done.


[]

In [27]:
%%sql
select * from rapecrimes
LIMIT 5;

5 rows affected.


[(datetime.datetime(1994, 8, 8, 18, 0), 'Monday   ', 'Aug ', 8, 1994, '18   ', 'RAPE ', 'A   ', 103, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 1041749, 196938, 1),
 (datetime.datetime(1995, 2, 1, 8, 0), 'Wednesday', 'Feb ', 1, 1995, '8    ', 'RAPE ', 'G   ', 83, 'BROOKLYN           ', 'N.Y. POLICE DEPT   ', 1006965, 193571, 2),
 (datetime.datetime(2000, 1, 9, 20, 0), 'Sunday   ', 'Jan ', 9, 2000, '20   ', 'RAPE ', 'G   ', 108, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 996793, 210001, 3),
 (datetime.datetime(2001, 1, 9, 15, 0), 'Tuesday  ', 'Jan ', 9, 2001, '15   ', 'RAPE ', 'H   ', 63, 'BROOKLYN           ', 'N.Y. HOUSING POLICE', 1000448, 168068, 4),
 (datetime.datetime(2001, 2, 11, 0, 1), 'Sunday   ', 'Feb ', 11, 2001, '0    ', 'RAPE ', 'B   ', 25, 'MANHATTAN          ', 'N.Y. POLICE DEPT   ', 1000555, 230994, 5)]

In [28]:
%%sql
SELECT * FROM rapecrimes 
ORDER BY time_id DESC
LIMIT 5;

5 rows affected.


[(datetime.datetime(2015, 11, 23, 3, 40), 'Monday   ', 'Nov ', 23, 2015, '3    ', 'RAPE ', 'C   ', 90, 'BROOKLYN           ', 'N.Y. POLICE DEPT   ', 997843, 196573, 13779),
 (datetime.datetime(2015, 11, 19, 23, 0), 'Thursday ', 'Nov ', 19, 2015, '23   ', 'RAPE ', 'K   ', 108, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 996793, 210001, 13778),
 (datetime.datetime(2015, 11, 19, 23, 0), 'Thursday ', 'Nov ', 19, 2015, '23   ', 'RAPE ', 'B   ', 67, 'BROOKLYN           ', 'N.Y. POLICE DEPT   ', 998032, 175598, 13777),
 (datetime.datetime(2015, 11, 19, 18, 0), 'Thursday ', 'Nov ', 19, 2015, '18   ', 'RAPE ', 'C   ', 103, 'QUEENS             ', 'N.Y. POLICE DEPT   ', 1041749, 196938, 13776),
 (datetime.datetime(2015, 11, 19, 3, 0), 'Thursday ', 'Nov ', 19, 2015, '3    ', 'RAPE ', 'E   ', 1, 'MANHATTAN          ', 'N.Y. POLICE DEPT   ', 982285, 201682, 13775)]

In [29]:
# To be continues, here we set a new column area_id that auto-increamented by 1 and starts from 15000. 
#Then we will have unique value for time_table and area_table seperately


Now create the table

In [30]:
%%sql
DROP TABLE IF EXISTS rapecrimes;
CREATE TABLE rapecrimes (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5),
    offense CHAR(5),
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19),
    xcor BIGINT,
    ycor BIGINT
    )

Done.
Done.


[]

create the table of dimension time, and create the primary key, indexing the table

In [31]:
%%sql
DROP TABLE IF EXISTS time_table;
CREATE TABLE time_table (
    date TIMESTAMP,
    day CHAR(9),
    month CHAR(4),
    day_n INT,
    year INT,
    hour CHAR(5)
    )

Done.
Done.


[]

In [32]:
!csvcut -c1,2,3,4,5,6 rape_final.csv > time.csv

In [33]:
!csvcut -n time.csv

  1: Occurrence Date
  2: Day of Week
  3: Occurrence Month
  4: Occurrence Day
  5: Occurrence Year
  6: Occurrence Hour


In [37]:
%%sql
COPY time_table FROM '/home/jovyan/work/time.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [38]:
%%sql
select * from time_table
LIMIT 5;

5 rows affected.


[(datetime.datetime(1994, 8, 8, 18, 0), 'Monday   ', 'Aug ', 8, 1994, '18   '),
 (datetime.datetime(1995, 2, 1, 8, 0), 'Wednesday', 'Feb ', 1, 1995, '8    '),
 (datetime.datetime(2000, 1, 9, 20, 0), 'Sunday   ', 'Jan ', 9, 2000, '20   '),
 (datetime.datetime(2001, 1, 9, 15, 0), 'Tuesday  ', 'Jan ', 9, 2001, '15   '),
 (datetime.datetime(2001, 2, 11, 0, 1), 'Sunday   ', 'Feb ', 11, 2001, '0    ')]

create the fact table of X and Y cordinate

create the table of AREA_TABLE, and create the primary key, indexing the table

In [58]:
!csvcut -c8,9,10,11 rape_final.csv > area.csv

In [59]:
%%sql
DROP TABLE IF EXISTS area_table;
CREATE TABLE area_table (
    sector CHAR(4),
    precint INT,
    borough CHAR(19),
    jur char(19)
    )

Done.
Done.


[]

In [60]:
%%sql
COPY area_table FROM '/home/jovyan/work/area.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [61]:
%%sql
select * from area_table
LIMIT 5;

5 rows affected.


[('A   ', 103, 'QUEENS             ', 'N.Y. POLICE DEPT   '),
 ('G   ', 83, 'BROOKLYN           ', 'N.Y. POLICE DEPT   '),
 ('G   ', 108, 'QUEENS             ', 'N.Y. POLICE DEPT   '),
 ('H   ', 63, 'BROOKLYN           ', 'N.Y. HOUSING POLICE'),
 ('B   ', 25, 'MANHATTAN          ', 'N.Y. POLICE DEPT   ')]

In [66]:
%%sql
DROP INDEX IF EXISTS area_key;
CREATE INDEX area_key ON area_table(precint);

Done.
Done.


[]

create the FACT_table , inseart the foreigh key of AREA and TIME

In [76]:
!csvcut -c1,9,12,13 rape_final.csv > fact.csv

In [77]:
%%sql
DROP TABLE IF EXISTS fact_table;
CREATE TABLE fact_table (
    date TIMESTAMP,
    precint INT,
    xcor BIGINT,
    ycor BIGINT
    )

Done.
Done.


[]

In [78]:
%%sql
COPY fact_table FROM '/home/jovyan/work/fact.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

13779 rows affected.


[]

In [79]:
%%sql
select * from fact_table
LIMIT 5;

5 rows affected.


[(datetime.datetime(1994, 8, 8, 18, 0), 103, 1041749, 196938),
 (datetime.datetime(1995, 2, 1, 8, 0), 83, 1006965, 193571),
 (datetime.datetime(2000, 1, 9, 20, 0), 108, 996793, 210001),
 (datetime.datetime(2001, 1, 9, 15, 0), 63, 1000448, 168068),
 (datetime.datetime(2001, 2, 11, 0, 1), 25, 1000555, 230994)]